In [1]:
import os, sys

PROJECT_ROOT = '/scratch/jq2uw/derm_vlms'
LLAVA_DERM_DIR = os.path.join(PROJECT_ROOT, 'llava_derm')

if LLAVA_DERM_DIR not in sys.path:
    sys.path.insert(0, LLAVA_DERM_DIR)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

import torch
torch.cuda.empty_cache()

from utils import load_model, predict_image, parse_label

print('Loading model...')
model, processor = load_model()
print('Model loaded.')

/home/jq2uw/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Loading model...


Loading checkpoint shards: 100%|████████████████████████████████████████| 3/3 [00:02<00:00,  1.11it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Total params: 7,063,427,072
Model loaded.


In [2]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path(PROJECT_ROOT) / 'data'

df = pd.read_parquet(os.path.join(PROJECT_ROOT, 'data_share', 'midas_share.parquet'))
print(f'Loaded {len(df)} rows')
print(f'y3 distribution:\n{df["y3"].value_counts()}')

def resolve_img_path(p):
    p = str(p)
    if os.path.isfile(p):
        return p
    candidate = DATA_DIR / Path(p).name
    if candidate.is_file():
        return str(candidate)
    return p

df['image_path_resolved'] = df['image_path'].apply(resolve_img_path)
n_found = df['image_path_resolved'].apply(os.path.isfile).sum()
print(f'Resolved images: {n_found}/{len(df)} found')

SEED = 42
N_PER_CLASS = 5
df_sample = df.groupby('y3', group_keys=False).apply(
    lambda g: g.sample(n=N_PER_CLASS, random_state=SEED),
).reset_index(drop=True)
print(f'\nStratified sample ({N_PER_CLASS} per class, seed={SEED}):')
print(df_sample['y3'].value_counts())
df_sample[['uid', 'y3', 'image_path_resolved']].head()

Loaded 3357 rows
y3 distribution:
y3
malignant    1391
benign       1322
other         644
Name: count, dtype: int64
Resolved images: 3357/3357 found

Stratified sample (5 per class, seed=42):
y3
benign       5
malignant    5
other        5
Name: count, dtype: int64


/scratch/jq2uw/conda_ex/tmp/ipykernel_868144/3641150031.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby('y3', group_keys=False).apply(


,uid,y3,image_path_resolved
0,1833,benign,/scratch/jq2uw/derm_vlms/data/s-prd-697891782.jpg
1,1191,benign,/scratch/jq2uw/derm_vlms/data/s-prd-593416010.jpg
2,610,benign,/scratch/jq2uw/derm_vlms/data/s-prd-639852881.jpg
3,1053,benign,/scratch/jq2uw/derm_vlms/data/s-prd-560547879.jpg
4,188,benign,/scratch/jq2uw/derm_vlms/data/s-prd-419238986.jpg


In [3]:
from PIL import Image
from tqdm import tqdm

q_describe = "Describe the lesion in detail."
q_classify = "Is the lesion malignant or benign, or other?"
q_describe_classify = q_describe + " " + q_classify
results = []

for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    uid = row['uid']
    try:
        image = Image.open(row['image_path_resolved']).convert('RGB')
    except Exception as e:
        print(f'[SKIP] uid={uid}: {e}')
        continue

    description = predict_image(model, processor, image, prompt=q_describe)
    classification = predict_image(model, processor, image, prompt=q_classify)
    describe_then_classify = predict_image(model, processor, image, prompt=q_describe_classify)

    results.append({
        'uid': uid,
        'ground_truth': row['y3'],
        'description': description,
        'classification': classification,
        'describe_then_classify': describe_then_classify,
    })

print(f'Collected {len(results)} predictions')

  0%|                                                                          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████| 15/15 [01:01<00:00,  4.11s/it]

Collected 15 predictions


In [4]:
results_df = pd.DataFrame(results)

RESULTS_DIR = os.path.join(LLAVA_DERM_DIR, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)
out_path = os.path.join(RESULTS_DIR, 'llava_derm_predictions.csv')
results_df.to_csv(out_path, index=False)
print(f'Saved {len(results_df)} rows to {out_path}')

results_df

Saved 15 rows to /scratch/jq2uw/derm_vlms/llava_derm/results/llava_derm_predictions.csv


,uid,ground_truth,description,classification,describe_then_classify
0,1833,benign,There is 1.0 chance that the lesion is Follicu...,There is 0.33 chance that the lesion is Atypic...,There is 0.5 chance that the lesion is Atypica...
1,1191,benign,There is 1.0 chance that the lesion is Purpura.,Is the lesion located on the skin?\n\nYes,There is 0.33 chance that the lesion is Purpur...
2,610,benign,There is 0.55 chance that the lesion is Tinea....,Is the lesion inflicted or unintentional?,There is 0.55 chance that the lesion is Pityri...
3,1053,benign,There is 0.55 chance that the lesion is Absces...,Is the lesion present in the skin of the abdomen?,There is 0.55 chance that the lesion is Cyst. ...
4,188,benign,There is 0.33 chance that the lesion is Melano...,Is the lesion a Dermatofibroma?,There is 0.55 chance that the lesion is Atypic...
5,3050,malignant,There is 0.55 chance that the lesion is Actini...,The lesion is benign.,There is 0.33 chance that the lesion is Basal ...
6,416,malignant,There is 0.5 chance that the lesion is Cutaneo...,The lesion is benign.,There is 1.0 chance that the lesion is benign.
7,3310,malignant,There is 0.55 chance that the lesion is Insect...,The lesion is benign.,There is 0.33 chance that the lesion is Impeti...
8,2450,malignant,There is 0.55 chance that the lesion is Hemang...,The lesion is benign.,There is 0.55 chance that the lesion is Hemang...
9,969,malignant,There is 0.55 chance that the lesion is Cutane...,Is the lesion infected?,There is 0.55 chance that the lesion is Basal ...
